In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from src.full_order_model import build_matrices, simulate_fom
from src.pod import compute_pod
from src.reduced_order import project_system, simulate_rom
from src.utils import ensure_results_dir



n = 20
t_eval = np.linspace(0, 10, 1000)

# -------------------------------
# Full Order Model
# -------------------------------
M, C, K = build_matrices(n)
X, t = simulate_fom(M, C, K, (0, 10), t_eval)

# -------------------------------
# POD
# -------------------------------
U, S, energy = compute_pod(X)

# --- Plot 1: Energy spectrum ---
plt.figure()
plt.plot(energy, marker="o")
plt.xlabel("Number of modes")
plt.ylabel("Cumulative energy")
plt.title("POD Energy Spectrum")
plt.grid(True)
plt.savefig(f"results/energy_spectrum.png", dpi=300, bbox_inches="tight")
plt.show()

# -------------------------------
# ROM Simulation
# -------------------------------
r = 5
Phi = U[:, :r]

Mr, Cr, Kr = project_system(M, C, K, Phi)
X_rom = simulate_rom(Mr, Cr, Kr, Phi, (0, 10), t_eval)

# --- Plot 2: FOM vs ROM ---
plt.figure()
plt.plot(t, X[0, :], label="FOM")
plt.plot(t, X_rom[0, :], "--", label=f"ROM (r={r})")
plt.xlabel("Time")
plt.ylabel("Displacement")
plt.legend()
plt.title("Full Order vs Reduced Order Response")
plt.grid(True)
plt.savefig(f"results/fom_vs_rom.png", dpi=300, bbox_inches="tight")
plt.show()

# -------------------------------
# Error vs Number of Modes
# -------------------------------
errors = []
max_modes = 10

for r in range(1, max_modes + 1):
    Phi = U[:, :r]
    Mr, Cr, Kr = project_system(M, C, K, Phi)
    Xr = simulate_rom(Mr, Cr, Kr, Phi, (0, 10), t_eval)

    error = np.linalg.norm(X - Xr) / np.linalg.norm(X)
    errors.append(error)

# --- Plot 3: Error vs modes ---
plt.figure()
plt.plot(range(1, max_modes + 1), errors, marker="o")
plt.xlabel("Number of POD modes")
plt.ylabel("Relative error")
plt.title("ROM Error vs Number of Modes")
plt.grid(True)
plt.savefig(f"results/error_vs_modes.png", dpi=300, bbox_inches="tight")
plt.show()

